# ケーススタディ４： アソシエーション分析（mlxtend）

In [291]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
online_retail = fetch_ucirepo(id=352) 
  
# data (as pandas dataframes) 
X = online_retail.data.features 
y = online_retail.data.targets 
  
# metadata 
print(online_retail.metadata) 
  
# variable information 
print(online_retail.variables) 

{'uci_id': 352, 'name': 'Online Retail', 'repository_url': 'https://archive.ics.uci.edu/dataset/352/online+retail', 'data_url': 'https://archive.ics.uci.edu/static/public/352/data.csv', 'abstract': 'This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.', 'area': 'Business', 'tasks': ['Classification', 'Clustering'], 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'], 'num_instances': 541909, 'num_features': 6, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': None, 'index_col': ['InvoiceNo', 'StockCode'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2015, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5BW33', 'creators': ['Daqing Chen'], 'intro_paper': {'title': 'Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining', 'aut

In [292]:
X.head() # X.shape == (541909, 6)

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [293]:
X.isnull().sum()

Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [294]:
X = X.dropna(subset=['Description'])
X = X.fillna(-1)
X.isnull().sum()

Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [302]:
# Extracting data by observing which country accounts for profit
X['Country'].value_counts().head(10)

Country
United Kingdom    494024
Germany             9495
France              8557
EIRE                8196
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         2002
Portugal            1519
Australia           1259
Name: count, dtype: int64

In [303]:
# Focus on "France", for instance, here
X1 = X[X['Country'] == 'France']
X2 = X1.drop('Country', axis=1)
X2.shape

(8557, 5)

In [304]:
X2['Description'].value_counts().head(10)

Description
POSTAGE                                311
RABBIT NIGHT LIGHT                      75
RED TOADSTOOL LED NIGHT LIGHT           72
PLASTERS IN TIN WOODLAND ANIMALS        68
PLASTERS IN TIN CIRCUS PARADE           68
ROUND SNACK BOXES SET OF4 WOODLAND      65
LUNCH BAG RED RETROSPOT                 61
LUNCH BOX WITH CUTLERY RETROSPOT        57
RED RETROSPOT MINI CASES                55
PLASTERS IN TIN SPACEBOY                55
Name: count, dtype: int64

In [327]:
X2.head(10), X2.shape

(                            Description  Quantity     InvoiceDate  UnitPrice  \
 26            ALARM CLOCK BAKELIKE PINK        24  12/1/2010 8:45       3.75   
 27            ALARM CLOCK BAKELIKE RED         24  12/1/2010 8:45       3.75   
 28           ALARM CLOCK BAKELIKE GREEN        12  12/1/2010 8:45       3.75   
 29      PANDA AND BUNNIES STICKER SHEET        12  12/1/2010 8:45       0.85   
 30                     STARS GIFT TAPE         24  12/1/2010 8:45       0.65   
 31          INFLATABLE POLITICAL GLOBE         48  12/1/2010 8:45       0.85   
 32   VINTAGE HEADS AND TAILS CARD GAME         24  12/1/2010 8:45       1.25   
 33      SET/2 RED RETROSPOT TEA TOWELS         18  12/1/2010 8:45       2.95   
 34  ROUND SNACK BOXES SET OF4 WOODLAND         24  12/1/2010 8:45       2.95   
 35                  SPACEBOY LUNCH BOX         24  12/1/2010 8:45       1.95   
 
     CustomerID  
 26     12583.0  
 27     12583.0  
 28     12583.0  
 29     12583.0  
 30     12583.0  

In [328]:
profit = X2['Quantity'] * X2['UnitPrice']

import pandas as pd

X3 = pd.concat([X2, profit], axis=1)

X4 = X3.drop('Quantity', axis=1).drop('UnitPrice', axis=1)
X4.columns = ['Description', 'InvoiceDate', 'CustomerID', 'Profit']

#w1 = X4.groupby('InvoiceDate', as_index=False)['Profit'].sum()
#w1 = X4.groupby(['InvoiceDate', 'CustomerID'])['Profit'].sum(numeric_only=True)
w1 = pd.DataFrame(X4.groupby(['InvoiceDate', 'Description'])['Profit'].sum()) # pd.DataFrame.groupbyのキーを２つ指定
#w1 = pd.DataFrame(X4.groupby(['InvoiceDate', 'CustomerID', 'Description'])['Profit'].sum()) # (InvoiceDate, CustomerID)で受注の一意性は確保できそうだが…

#w1.head(), w1.shape
w1

Profit
InvoiceDate     Description                             
1/10/2011 13:22 ALARM CLOCK BAKELIKE CHOCOLATE     30.00
                ALARM CLOCK BAKELIKE GREEN         30.00
                ALARM CLOCK BAKELIKE IVORY         30.00
                ALARM CLOCK BAKELIKE ORANGE        30.00
                ALARM CLOCK BAKELIKE PINK          30.00
...                                                  ...
9/9/2011 8:48   POLYESTER FILLER PAD 40x40cm       17.40
                RED KITCHEN SCALES                 17.00
                RED RETROSPOT PEG BAG              30.60
                RETROSPOT HEART HOT WATER BOTTLE   14.85
                SCOTTIE DOG HOT WATER BOTTLE       29.70

[8519 rows x 1 columns]

In [330]:
w2 = w1.unstack().reset_index().fillna(0).set_index('InvoiceDate')
w2.head(), w2.shape

(                                        Profit                     \
 Description      50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER   
 InvoiceDate                                                         
 1/10/2011 13:22                            0.0                0.0   
 1/11/2011 11:48                            0.0                0.0   
 1/11/2011 13:31                            0.0                0.0   
 1/11/2011 15:00                            0.0                0.0   
 1/11/2011 7:57                             0.0                0.0   
 
                                                                        \
 Description      I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY   
 InvoiceDate                                                             
 1/10/2011 13:22                          0.0                      0.0   
 1/11/2011 11:48                          0.0                      0.0   
 1/11/2011 13:31                          0.0                      0

In [331]:
basket_df = w2.apply(lambda x: x > 0)

# 縦・横＝イベント記録（時刻、場所などの識別子）・イベント名のテーブルが必要となる
basket_df

Profit                     \
Description      50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER   
InvoiceDate                                                         
1/10/2011 13:22                          False              False   
1/11/2011 11:48                          False              False   
1/11/2011 13:31                          False              False   
1/11/2011 15:00                          False              False   
1/11/2011 7:57                           False              False   
...                                        ...                ...   
9/8/2011 10:32                           False              False   
9/8/2011 10:35                           False               True   
9/9/2011 10:51                           False              False   
9/9/2011 11:48                           False               True   
9/9/2011 8:48                            False              False   

                                                                       \
Description      I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY   
InvoiceDate                                                             
1/10/2011 13:22                        False                    False   
1/11/2011 11:48                        False                    False   
1/11/2011 13:31                        False                    False   
1/11/2011 15:00                        False                    False   
1/11/2011 7:57                         False                    False   
...                                      ...                      ...   
9/8/2011 10:32                         False                    False   
9/8/2011 10:35                         False                    False   
9/9/2011 10:51                         False                    False   
9/9/2011 11:48                         False                    False   
9/9/2011 8:48                          False                    False   

                                                                          \
Description      SET 2 TEA TOWELS I LOVE LONDON   SPACEBOY BABY GIFT SET   
InvoiceDate                                                                
1/10/2011 13:22                            False                   False   
1/11/2011 11:48                            False                   False   
1/11/2011 13:31                            False                   False   
1/11/2011 15:00                            False                   False   
1/11/2011 7:57                             False                   False   
...                                          ...                     ...   
9/8/2011 10:32                             False                   False   
9/8/2011 10:35                             False                   False   
9/9/2011 10:51                             False                   False   
9/9/2011 11:48                             False                   False   
9/9/2011 8:48                              False                   False   

                                                           \
Description      TRELLIS COAT RACK 10 COLOUR SPACEBOY PEN   
InvoiceDate                                                 
1/10/2011 13:22              False                  False   
1/11/2011 11:48              False                  False   
1/11/2011 13:31              False                  False   
1/11/2011 15:00              False                   True   
1/11/2011 7:57               False                  False   
...                            ...                    ...   
9/8/2011 10:32               False                  False   
9/8/2011 10:35               False                  False   
9/9/2011 10:51               False                   True   
9/9/2011 11:48               False                  False   
9/9/2011 8:48                False                  False   

                                                                      ...  \
Description     12 COLOURED PARTY BALLOONS 12 EGG HOU

In [332]:
from mlxtend.frequent_patterns import apriori, association_rules

In [343]:
freq_items1 = apriori(basket_df, min_support=0.1, use_colnames=True)
print(freq_items1.sort_values('support', ascending=False).head(10))
freq_items1.shape[0]

     support                                           itemsets
7   0.650759                                ((Profit, POSTAGE))
8   0.160521                     ((Profit, RABBIT NIGHT LIGHT))
10  0.154013          ((Profit, RED TOADSTOOL LED NIGHT LIGHT))
6   0.145336       ((Profit, PLASTERS IN TIN WOODLAND ANIMALS))
4   0.143167         ((Profit, PLASTERS IN TIN CIRCUS PARADE ))
21  0.140998  ((Profit, POSTAGE), (Profit, RABBIT NIGHT LIGHT))
12  0.134490    ((Profit, ROUND SNACK BOXES SET OF4 WOODLAND ))
22  0.134490  ((Profit, POSTAGE), (Profit, RED TOADSTOOL LED...
1   0.130152                ((Profit, LUNCH BAG RED RETROSPOT))
23  0.125813  ((Profit, ROUND SNACK BOXES SET OF4 WOODLAND )...


25

In [344]:
a_rules1 = association_rules(freq_items1, metric='lift', min_threshold=1)

a_rules1 = a_rules1.sort_values('lift', ascending=False).reset_index(drop=True)

print(a_rules1.head(5))

a_rules1.shape[0]

                                       antecedents  \
0        ((Profit, SET/6 RED SPOTTY PAPER PLATES))   
1          ((Profit, SET/6 RED SPOTTY PAPER CUPS))   
2  ((Profit, ROUND SNACK BOXES SET OF4 WOODLAND ))   
3                              ((Profit, POSTAGE))   
4       ((Profit, PLASTERS IN TIN CIRCUS PARADE ))   
5                              ((Profit, POSTAGE))   
6                              ((Profit, POSTAGE))   
7                   ((Profit, RABBIT NIGHT LIGHT))   
8                              ((Profit, POSTAGE))   
9        ((Profit, RED TOADSTOOL LED NIGHT LIGHT))   

                                       consequents  antecedent support  \
0          ((Profit, SET/6 RED SPOTTY PAPER CUPS))            0.108460   
1        ((Profit, SET/6 RED SPOTTY PAPER PLATES))            0.117137   
2                              ((Profit, POSTAGE))            0.134490   
3  ((Profit, ROUND SNACK BOXES SET OF4 WOODLAND ))            0.650759   
4                              ((Pr

14